**在Tensor中的设置与属性**

In [26]:
import torch
x = torch.ones(2, 2, requires_grad=True)
# 将其属性.requires_grad设置为True，它将开始追踪(track)在其上的所有操作
print(x)
print(x.grad_fn)
# 每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是
# 通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。
# 像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [27]:
y = x + 2
print(y)
print(y.grad_fn)



tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [28]:
z = y*y + 2
print(z)
z = z.mean()
print(z)


tensor([[11., 11.],
        [11., 11.]], grad_fn=<AddBackward0>)
tensor(11., grad_fn=<MeanBackward0>)


In [29]:
print(z.grad_fn)

改变requis_grad的属性

In [30]:
a = torch.randn(2,2)    #缺失情况下默认 requires_grad = False
print(a.requires_grad)




False


In [31]:
a.requires_grad_(True)
print(a.requires_grad)



True


##梯度##

In [32]:
z.backward()
print(x.grad)
# z关于x的梯度
print(y.grad)


tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])
None


In [34]:
out = x.sum()
out.backward()
print(x.grad)
out1 = x.sum()
out.backward()
print(x.grad)
# 每次反向传播,梯度是累加的


tensor([[3.5000, 3.5000],
        [3.5000, 3.5000]])
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [35]:
out2 = x.sum()
x.grad.data.zero_()
# 将梯度清零
out.backward()
print(x.grad)



tensor([[1., 1.],
        [1., 1.]])


**当输出不是标量时**

In [36]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)



tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


由于z不是标量,无法对由向量形式的自变量进行求导,所以要将z加权求和转换为一个标量<br/>
因此,在调用backward时需要传入一个和z同形的权重向量

In [37]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)




tensor([2.0000, 0.2000, 0.0200, 0.0020])


**中断梯度追踪**

In [38]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x**2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
print(x.requires_grad)
print(y1.requires_grad, y2.requires_grad, y3.requires_grad)





True
True False True


In [39]:
y3.backward()
print(x.grad)
# 没有对y2进行梯度追踪,所以程序将y2当做常数进行处理,即y2对应的梯度为0

tensor(2.)
